In [98]:
import json
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import Dict, Any

In [99]:
load_dotenv()

True

In [100]:
# Información de la empresa (configurable)
EMPRESA_INFO = {
    "nombre": "Componentes Intergalácticos Industriales S.A.",
    "cargo_responsable": "Responsable de Atención al Cliente",
    "nombre_responsable": "Nebula Prime",
    "contacto": {
        "email": "devoluciones@ciindustriales.com",
        "holofono": "+34 9X9 000 001",
        "horario": "Lunes a Viernes de 8:00 a 18:00"
    }
}

In [101]:
# 1. Función para extraer información del email (sin decorador @tool)
def extractor_info_devolucion(email_content: str) -> Dict[str, Any]:
    """Extrae información clave de un email de solicitud de devolución."""
    prompt = f"""Extrae esta información del email y devuelve SOLO un JSON válido:
    {{
        "número_de_pedido": "str",
        "nombre_cliente": "str",
        "departamento_cliente": "str",
        "producto_solicitado": "str",
        "motivo_solicitud": "str (10 palabras max)",
        "evidencia_adjunta": "str (ej: 'sí: imágenes' o 'no')",
        "urgencia": "str (alta/media/baja)"
    }}

    Email: {email_content}"""
    
    llm = ChatOpenAI(
        temperature=0,
        model="gpt-4o",
        model_kwargs={"response_format": {"type": "json_object"}}
    )
    
    response = llm.invoke(prompt)
    
    try:
        content = response.content.strip()
        if content.startswith("```json"):
            content = content[7:-3].strip()
        elif content.startswith("```"):
            content = content[3:-3].strip()
        
        return json.loads(content)
    except Exception as e:
        print(f"Error procesando respuesta: {e}")
        return {"error": str(e)}


In [102]:
# 2. Función para evaluar la solicitud (sin decorador @tool)
def evaluador_devolucion(info_solicitud: Dict[str, Any]) -> Dict[str, Any]:
    """Evalúa si una solicitud debe ser aceptada según políticas."""
    required_fields = [
        "número_de_pedido", "nombre_cliente", "departamento_cliente",
        "producto_solicitado", "motivo_solicitud", "evidencia_adjunta", "urgencia"
    ]
    
    for field in required_fields:
        if field not in info_solicitud:
            raise ValueError(f"Campo requerido faltante: {field}")
    
    prompt = f"""Evalúa esta solicitud según políticas y devuelve SOLO un JSON válido:
    {{
        "motivo_aceptable": "bool",
        "motivo_rechazo": "bool",
        "evidencia_suficiente": "bool",
        "decision_preliminar": "str (aceptar/rechazar)",
        "razon": "str (50 palabras max)"
    }}

    Políticas:
    Aceptar: Defecto fabricación, Error suministro, Producto incompleto
    Rechazar: Daños transporte no asegurado, Manipulación cliente, Plazo superado
    
    Solicitud: {json.dumps(info_solicitud, ensure_ascii=False)}"""
    
    llm = ChatOpenAI(
        temperature=0,
        model="gpt-4o",
        model_kwargs={"response_format": {"type": "json_object"}}
    )
    
    response = llm.invoke(prompt)
    return json.loads(response.content)


In [103]:
# 3. Función para generar respuesta
def generador_respuesta(info_solicitud: Dict[str, Any], evaluacion: Dict[str, Any]) -> str:
    """Genera una respuesta profesional al cliente."""
    prompt = f"""Redacta un email de respuesta profesional (100-150 palabras) con esta estructura:

    Asunto: Respuesta a solicitud de devolución - Pedido {info_solicitud['número_de_pedido']}

    Estimado/a {info_solicitud['nombre_cliente']},

    [Texto generado con: agradecimiento, resumen evaluación, comunicación decisión, alternativas si es rechazo]

    Para cualquier consulta adicional, no dude en contactarnos.

    Atentamente,
    {EMPRESA_INFO['nombre_responsable']}
    {EMPRESA_INFO['cargo_responsable']}
    {EMPRESA_INFO['nombre']}
    
    Contacto:
    - Email: {EMPRESA_INFO['contacto']['email']}
    - Holofono: {EMPRESA_INFO['contacto']['holofono']}
    - Horario de atención: {EMPRESA_INFO['contacto']['horario']}

    Info solicitud: {json.dumps(info_solicitud, ensure_ascii=False)}
    Evaluación: {json.dumps(evaluacion, ensure_ascii=False)}"""
    
    llm = ChatOpenAI(
        temperature=0.7,
        model="gpt-4o"
    )
    return llm.invoke(prompt).content


In [104]:
# Procesamiento completo simplificado
def procesar_devolucion(email_content: str) -> Dict[str, Any]:
    """Procesa una solicitud de devolución completa."""
    resultados = {}
    
    # 1. Extraer información
    resultados["info"] = extractor_info_devolucion(email_content)
    if "error" in resultados["info"]:
        raise ValueError(f"Error en extracción: {resultados['info']['error']}")
    
    # 2. Evaluar solicitud
    resultados["evaluacion"] = evaluador_devolucion(resultados["info"])
    
    # 3. Generar respuesta
    resultados["respuesta"] = generador_respuesta(
        resultados["info"],
        resultados["evaluacion"]
    )
    
    return resultados

In [105]:
# Email de ejemplo (podría venir de un parámetro o lectura de archivo)
email_ejemplo = """
Asunto: Solicitud de reemplazo por daños en transporte  Pedido #D347-STELLA
Estimado equipo de Componentes Intergalácticos Industriales S.A.,
Me pongo en contacto con ustedes como cliente reciente para comunicar una incidencia relacionada con el pedido #D347-STELLA, correspondiente a un lote de condensadores de fluzo modelo FX-88, destinados a un proyecto estratégico de gran envergadura: la construcción de la Estrella de la Muerte.
Lamentablemente, al recibir el envío, observamos que varios de los condensadores presentaban daños visibles y no funcionales. Tras revisar el estado del embalaje y consultar con el piloto de carga, todo indica que la mercancía sufrió una caída durante el transporte interestelar.
Dado que estos componentes son críticos para la activación del núcleo central del sistema de rayos destructores, les solicitamos con carácter urgente el reemplazo inmediato de las unidades defectuosas, así como una revisión de los protocolos de embalaje y transporte para evitar que algo así vuelva a ocurrir.
Adjunto imágenes del estado de los condensadores y el albarán de entrega sellado por nuestro droide de recepción.
Agradezco de antemano su pronta atención a este asunto. Quedamos a la espera de su respuesta para coordinar el reemplazo.
Atentamente,
Darth Márquez
Departamento de Ingeniería Imperial
Sector de Proyectos Especiales
Contacto: dmarquez@imperiumgalactic.net
Holofono: +34 9X9 123 456
"""

In [106]:
try:
    resultado = procesar_devolucion(email_ejemplo)
    print("\nInformación extraída:")
    print(json.dumps(resultado["info"], indent=2, ensure_ascii=False))
    print("\nEvaluación:")
    print(json.dumps(resultado["evaluacion"], indent=2, ensure_ascii=False))
    print("\nRespuesta generada:")
    print(resultado["respuesta"])
except Exception as e:
    print(f"Error durante el procesamiento: {e}")
    import traceback
    traceback.print_exc()


Información extraída:
{
  "número_de_pedido": "D347-STELLA",
  "nombre_cliente": "Darth Márquez",
  "departamento_cliente": "Departamento de Ingeniería Imperial",
  "producto_solicitado": "condensadores de fluzo modelo FX-88",
  "motivo_solicitud": "daños visibles y no funcionales",
  "evidencia_adjunta": "sí: imágenes",
  "urgencia": "alta"
}

Evaluación:
{
  "motivo_aceptable": false,
  "motivo_rechazo": true,
  "evidencia_suficiente": true,
  "decision_preliminar": "rechazar",
  "razon": "El motivo de la solicitud no coincide con las políticas de aceptación establecidas."
}

Respuesta generada:
Asunto: Respuesta a solicitud de devolución - Pedido D347-STELLA

Estimado Darth Márquez,

Gracias por comunicarse con nosotros respecto a su solicitud de devolución del pedido D347-STELLA. Tras una evaluación detallada de su caso, hemos analizado las imágenes proporcionadas y revisado el motivo de su solicitud. Lamentamos informarle que no podemos proceder con la devolución, ya que el motiv